# Lab 2

## Task 05: ConvNet CIFAR-10

## CIFAR-10

In [1]:
import tensorflow as tf        
# to visualize the resutls
import matplotlib.pyplot as plt 
# 70k mnist dataset that comes with the tensorflow container
from tensorflow.examples.tutorials.mnist import input_data      

tf.set_random_seed(0)   

In [2]:
import cPickle
import os
import os.path
import numpy as np

def one_hot_encode(values, size):
    b = np.zeros((values.shape[0], size))
    b[np.arange(values.shape[0]), values] = 1
    return b

with open('data/cifar-10-batches-py/test_batch') as fp:
    CF10_test = cPickle.load(fp)
    
CF10_X_test = np.array(CF10_test['data'])
CF10_Y_test = np.array(CF10_test['labels'])

In [3]:
path = 'data/cifar-10-batches-py/'
CF10_X = None
CF10_Y = None

for filename in [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]:
    if 'data_batch' in filename:
        with open(filename, 'r') as fp:
            print(filename)
            d = cPickle.load(fp)
            if CF10_X is not None:
                CF10_X = np.concatenate((CF10_X, d['data']), axis=0)
                CF10_Y = np.concatenate((CF10_Y, d['labels']), axis=0)
            else:
                CF10_X = d['data']
                CF10_Y = d['labels']

CF10_Y = one_hot_encode(CF10_Y, 10)
CF10_Y_test = one_hot_encode(CF10_Y_test, 10)

print "X_train: %s, Y_train: %s" % (CF10_X.shape, CF10_Y.shape)
print "X_test: %s, Y_test: %s" % (CF10_X_test.shape, CF10_Y_test.shape)



data/cifar-10-batches-py/data_batch_1
data/cifar-10-batches-py/data_batch_2
data/cifar-10-batches-py/data_batch_5
data/cifar-10-batches-py/data_batch_3
data/cifar-10-batches-py/data_batch_4
X_train: (50000, 3072), Y_train: (50000, 10)
X_test: (10000, 3072), Y_test: (10000, 10)


In [4]:
def randombatch(n):
    rows = np.random.randint(CF10_X.shape[0], size=n)    
    return CF10_X[rows, :], CF10_Y[rows, :]

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, strides):
    return tf.nn.conv2d(x, W, strides=strides, padding='SAME')

In [ ]:
def runtest():
    
    acc, cc = [], []
    for i in range(0, CF10_X_test.shape[0], 500):
        rows = np.array(range(i, i+500))
    
        a, c = sess.run([accuracy, cross_entropy], feed_dict={X: CF10_X_test[rows, :], Y_: CF10_Y_test[rows, :],  pkeep: 1.0})
        acc.append(a)
        cc.append(c)
    
    _accuracy, _loss = sum(acc)/len(acc), sum(cc)/len(cc)
    return _accuracy, _loss


In [ ]:
                                        
# load data, 60K trainset and 10K testset
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)  

# 1. Define Variables and Placeholders


X = tf.placeholder(tf.float32, [None, 3072])
Y_ = tf.placeholder(tf.float32, [None, 10])
XX = tf.reshape(X, [-1, 32, 32, 3])
pkeep = tf.placeholder(tf.float32)

# 2. Define the model

# Stride of 2 reduces WxH by 2?

global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.005
lr = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100000, 0.96, staircase=False)

# we get one vector from one conv layer, and pass it to the next one to transform it
W1 = tf.Variable(tf.truncated_normal([5, 5, 3, 12], stddev=0.1))
B1 = tf.Variable(tf.zeros([12]))
W1CONV = tf.nn.relu(conv2d(XX, W1, strides=[1,1,1,1]) + B1)
W1CONVD = tf.nn.dropout(W1CONV, pkeep)


W2 = tf.Variable(tf.truncated_normal([5, 5, 12, 8], stddev=0.1))
B2 = tf.Variable(tf.zeros([8]))
W2CONV = tf.nn.relu(conv2d(W1CONVD, W2, strides=[1,2,2,1]) + B2)
W2CONVD = tf.nn.dropout(W2CONV, pkeep)


W3 = tf.Variable(tf.truncated_normal([5, 5, 8, 12], stddev=0.1))
B3 = tf.Variable(tf.zeros([12]))
W3CONV = tf.nn.relu(conv2d(W2CONVD, W3, strides=[1,2,2,1]) + B3)
W3CONVD = tf.nn.dropout(W3CONV, pkeep)

# print W1CONV.get_shape()
# print W2CONV.get_shape()
# print W3CONV.get_shape()

W4 = weight_variable([8 * 8 * 12, 200])
B4 = tf.Variable(tf.zeros([200]))
W4FC = tf.nn.relu(tf.matmul(tf.reshape(W3CONVD, [-1, 8 * 8 * 12]), W4) + B4)
W4FC = tf.nn.dropout(W4FC, pkeep)

W5 = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1))
B5 = tf.Variable(tf.zeros([10]))

Ylogits = tf.nn.relu(tf.matmul(W4FC, W5) + B5)
Y = tf.nn.softmax(Ylogits)

# # 3. Define the loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(Y, Y_))

# # 4. Define the accuracy 
accuracy = tf.Variable(tf.zeros([1], tf.float32))
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# # 5. Define an optimizer
# train_step = tf.train.GradientDescentOptimizer(lr).minimize(cross_entropy, global_step=global_step)
train_step = tf.train.MomentumOptimizer(lr, 10e-8).minimize(cross_entropy, global_step=global_step)

# # initialize
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)


def training_step(i, update_test_data, update_train_data):

    print "\r", i,
    ####### actual learning 
    # reading batches of 100 images with 100 labels
#     batch_X, batch_Y = mnist.train.next_batch(100)
    batch_X, batch_Y = randombatch(100)
    # the backpropagation training step
    sess.run(train_step, feed_dict={X: batch_X, Y_: batch_Y,  pkeep: .6})
    
    ####### evaluating model performance for printing purposes
    # evaluation used to later visualize how well you did at a particular time in the training
    train_a = []
    train_c = []
    test_a = []
    test_c = []
    if update_train_data:
        a, c = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, Y_: batch_Y, pkeep: .6})
        train_a.append(a)
        train_c.append(c)
        print "TrainA[%f], TrainC[%f]" % (a, c)

    if update_test_data:
#         a, c = sess.run([accuracy, cross_entropy], feed_dict={X: CF10_X_test, Y_: CF10_Y_test,  pkeep: 1.0})
        a, c = runtest()
        test_a.append(a)
        test_c.append(c)
        print "TestA[%f], TestC[%f]" % (a, c)

    
    return (train_a, train_c, test_a, test_c)


# 6. Train and test the model, store the accuracy and loss per iteration

train_a = []
train_c = []
test_a = []
test_c = []
    
training_iter = 10000
epoch_size = 100
for i in range(training_iter):
    test = False
    if i % epoch_size == 0:
        test = True
    a, c, ta, tc = training_step(i, test, test)
    
    train_a += a
    train_c += c
    test_a += ta
    test_c += tc


0 TrainA[0.110000], TrainC[2.351152]
TestA[0.112200], TestC[2.346878]
100 TrainA[0.140000], TrainC[2.321150]
TestA[0.100000], TestC[2.361145]
200 TrainA[0.070000], TrainC[2.391150]
TestA[0.100000], TestC[2.361150]
300 TrainA[0.160000], TrainC[2.299074]
TestA[0.100000], TestC[2.361150]
400 TrainA[0.150000], TrainC[2.311101]
TestA[0.100000], TestC[2.361150]
500 TrainA[0.120000], TrainC[2.341150]
TestA[0.100000], TestC[2.361150]
600 TrainA[0.120000], TrainC[2.341150]
TestA[0.100000], TestC[2.361150]
700 TrainA[0.100000], TrainC[2.361150]
TestA[0.100000], TestC[2.361150]
800 TrainA[0.090000], TrainC[2.371150]
TestA[0.100000], TestC[2.361150]
900 TrainA[0.080000], TrainC[2.381150]
TestA[0.100000], TestC[2.361150]
1000 TrainA[0.130000], TrainC[2.331150]
TestA[0.100000], TestC[2.361150]
1100 TrainA[0.080000], TrainC[2.381150]
TestA[0.100000], TestC[2.361150]
1200 TrainA[0.080000], TrainC[2.381150]
TestA[0.100000], TestC[2.361150]
1300 TrainA[0.140000], TrainC[2.321150]
TestA[0.100000], TestC[

In [ ]:
# 7. Plot and visualise the accuracy and loss

# accuracy training vs testing dataset
plt.plot(train_a, label="Train A")
plt.plot(test_a, label="Test A")
plt.grid(True)
plt.legend(bbox_to_anchor=(0, 1), loc='lower center', ncol=1)
plt.show()


In [ ]:
# loss training vs testing dataset
plt.plot(train_c, label="Train C")
plt.plot(test_c, label="Test C")
plt.grid(True)
plt.legend(bbox_to_anchor=(0, 1), loc='lower center', ncol=1)
plt.show()


In [ ]:
# Zoom in on the tail of the plots
zoom_point = 50
x_range = range(zoom_point,training_iter/epoch_size)
plt.plot(x_range, train_a[zoom_point:], label="Train A")
plt.plot(x_range, test_a[zoom_point:], label="Test A")
plt.grid(True)
plt.legend(bbox_to_anchor=(0, 1), loc='lower center', ncol=1)
plt.show()

In [ ]:
plt.plot(x_range, train_c[zoom_point:], label="Train C")
plt.plot(x_range, test_c[zoom_point:], label="Test C")
plt.grid(True)
plt.legend(bbox_to_anchor=(0, 1), loc='lower center', ncol=1)
plt.show()

In [ ]:
# a, c = sess.run([accuracy, cross_entropy], feed_dict={X: CF10_X_test, Y_: CF10_Y_test,  pkeep: 1.0})
a, c = runtest()
print "Accuracy: %f, Loss: %f" %(a, c)

Testing performs better than training. Best results with dropout .6-.65 instead of .5

In [ ]:
sess.close()